In [ ]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model = "llama3.2:1b")

In [ ]:
#PDF Loader
from langchain.document_loaders import PyPDFLoader
file_path = './Learning_Python.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()

In [ ]:
#Text chunk creation
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=650, chunk_overlap=0)
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks)

In [ ]:
#Creating Embeddings for each of the chunk
from langchain_ollama import OllamaEmbeddings
embed = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)

In [ ]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
vector_store = Chroma(
    collection_name="PythonBook",
    embedding_function=embed,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

vector_store.add_documents(chunks)

In [ ]:
retriever = vector_store.as_retriever()
#retreived_docs = retriever.invoke("What are Lists?")

In [77]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Explain the question in detail with examples.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nExplain the question in detail with examples.\nQuestion: {question} \nContext: {context} \nAnswer:\n"), additional_kwargs={})]


In [79]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "What is the difference between classes and dictionaries?"
rag_chain.invoke(query)

'I\'d be happy to help you understand the difference between classes and dictionaries.\n\n**What is a Class?**\n\nA class is a blueprint or a template that defines the properties and behavior of an object. It\'s essentially a template for creating objects that have similar characteristics. A class typically includes attributes (data) and methods (functions that operate on that data). When you create an instance of a class, you\'re creating a new object that has its own set of attributes and can perform its specific actions.\n\n**What is a Dictionary?**\n\nA dictionary is a collection of key-value pairs where each key is unique and maps to a specific value. In other words, dictionaries are collections of data with keys that identify the values. Dictionaries are mutable (meaning they can be changed after creation), meaning you can add or remove items from them as needed.\n\n**Key differences between Classes and Dictionaries:**\n\n1. **Data storage**: A class stores attributes (data) whil

In [73]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()


D:\Github Projects\RAG\Local_RAG_LLama3.1\myenv\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [75]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is the difference between classes and dictionaries"):
    print(chunk, end="", flush=True)

Here's the answer to your question:

A class is a collection of attributes (data) and methods (logic), whereas a dictionary is an unordered collection of key-value pairs. The main difference between them is that classes can store multiple instances with different attribute names, while dictionaries are indexed by keys, not positions.

Additionally, classes have more flexibility in terms of their structure and organization, allowing for the creation of nested data structures or inheritance hierarchies. On the other hand, dictionaries are typically simpler and more straightforward in their implementation.